In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import re
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [20]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [21]:
environment = jinja2.Environment()

In [58]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [23]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [59]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [60]:
typespec

{'reasoning': 'For a gym tracking bot, we need to handle several key aspects:\n1. Recording exercises with details like sets, reps, weight\n2. Equipment availability to suggest appropriate exercises\n3. Time constraints for workout planning\n4. Progress tracking over time\n\nExpected user messages:\n"I did 3 sets of bench press with 165lbs, 12 reps each"\n-> recordExercise({name: "bench press", sets: 3, reps: 12, weight: 165})\n\n"Show me my chest progress for last month"\n-> getProgress({muscleGroup: "chest", from: [date], to: [date]})\n\n"Create a 45-minute workout plan. I have access to dumbbells and a bench"\n-> planWorkout({duration: 45, equipment: ["dumbbells", "bench"]})',
 'typespec_definitions': 'model Exercise {\n  name: String\n  sets: Int32\n  reps: Int32\n  weight: Float32\n  date: DateTime\n}\n\nmodel Equipment {\n  name: String\n  type: String\n}\n\nmodel WorkoutPlan {\n  duration: Int32\n  exercises: PlannedExercise[]\n}\n\nmodel PlannedExercise {\n  name: String\n  sug

In [26]:
print(typespec["reasoning"])

For a gym tracking bot, users will likely send messages like:
"I did 3 sets of bench press with 80kg" or "Today I worked out for 45 minutes doing squats and deadlifts"
or "Suggest a 30-minute workout for chest using dumbbells and bench"

Key functions needed:
1. Record exercise with sets, reps, and weight
2. Log workout session with duration and exercises
3. Get workout suggestions based on:
   - Available equipment
   - Target muscle groups
   - Time constraint
4. Track progress over time

The LLM can extract these details from natural language:
"I did 3 sets of bench press with 80kg" -> 
recordExercise({
    name: "bench press",
    sets: 3,
    weight: 80,
    muscleGroup: "chest"
})

"Suggest a 30-minute workout for chest using dumbbells and bench" ->
suggestWorkout({
    duration: 30,
    equipment: ["dumbbells", "bench"],
    targetMuscle: "chest"
})


In [27]:
print(typespec["typespec_definitions"])

model Exercise {
    name: string;
    sets: int32;
    reps: int32;
    weight?: float32;
    muscleGroup: string;
}

model WorkoutSession {
    date: Date;
    duration: int32;
    exercises: Exercise[];
}

model Equipment {
    name: string;
    type: string;
}

interface GymBot {
    @llm_func(2)
    recordExercise(exercise: Exercise): void;

    @llm_func(2)
    logWorkoutSession(session: WorkoutSession): void;

    @llm_func(3)
    suggestWorkout(duration: int32, equipment: Equipment[], targetMuscle: string): Exercise[];

    @llm_func(1)
    getProgress(exerciseName: string, from: Date, to: Date): Exercise[];
}


In [28]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [29]:
print(drizzle["drizzle_schema"])

import { serial, text, integer, real, timestamp, pgTable } from "drizzle-orm/pg-core";

export const exercisesTable = pgTable("exercises", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight"),
  muscle_group: text("muscle_group").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const workoutSessionsTable = pgTable("workout_sessions", {
  id: serial("id").primaryKey(),
  date: timestamp("date").notNull(),
  duration: integer("duration").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  type: text("type").notNull(),
  created_at: timestamp("created_at").defaultNow().notNull()
});

// Junction table for workout sessions and exercises
export const workoutExercisesTable = pgTable("workout_exercises", {
  

In [30]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [31]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise with details like name, sets, reps, weight (optional), and muscle group targeted',
  'examples': ['I did 3 sets of 12 bench presses with 135 pounds',
   'Just completed 4 sets of pullups, 8 reps each',
   'Recorded 3x10 squats targeting legs',
   'Add my shoulder press exercise: 3 sets, 10 reps, 45 pounds']},
 {'name': 'logWorkoutSession',
  'description': 'Records a complete workout session including multiple exercises, duration, and date',
  'examples': ['I want to record my exercise routine for today',
   'Log my workout session from this morning',
   'Save my 45-minute workout with these exercises',
   'Record my gym session with the following exercises']},
 {'name': 'suggestWorkout',
  'description': 'Suggests exercises based on available equipment, desired duration, and target muscle group',
  'examples': ['What exercises can I do for chest with dumbbells in 30 minutes?',
   'Suggest a back workout using resi

In [32]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [33]:
print(pre_response.content[0].text)

<instructions>
For the getProgress function, parse user input to extract:
1. Exercise name (required) - default to exact match from input
2. From date (required) - if not specified, default to 30 days ago from current date
3. To date (required) - if not specified, default to current date
Dates should be in ISO 8601 format (YYYY-MM-DD)
Exercise names should be normalized to lower case
</instructions>

<examples>
    <example>
        <input>Show me my bench press progress since January</input>
        <output>{
            "exerciseName": "bench press",
            "from": "2024-01-01",
            "to": "2024-02-14"
        }</output>
    </example>

    <example>
        <input>How am I doing with squats from last week?</input>
        <output>{
            "exerciseName": "squats",
            "from": "2024-02-07",
            "to": "2024-02-14"
        }</output>
    </example>

    <example>
        <input>deadlift progress between March 1 and April 15 2023</input>
        <output>

In [34]:
pre_processors

{'recordExercise': {'instructions': 'For recordExercise function, parse user input to extract exercise details:\n1. Name should be a recognizable exercise name\n2. Sets and reps are required integers\n3. Weight is optional float in kilograms/pounds\n4. Muscle group should be inferred from exercise name if not specified\n5. Handle variations of input format (text description, shorthand notation)',
  'examples': [('I did bench press 3 sets of 8 reps with 80kg',
    '{\n    "name": "bench press",\n    "sets": 3,\n    "reps": 8,\n    "weight": 80.0,\n    "muscleGroup": "chest"\n}'),
   ('bench press 80x5x3',
    '{\n    "name": "bench press",\n    "sets": 3,\n    "reps": 5,\n    "weight": 80.0,\n    "muscleGroup": "chest"\n}'),
   ('did 3 sets of pull-ups, 12 reps each',
    '{\n    "name": "pull-ups",\n    "sets": 3,\n    "reps": 12,\n    "weight": null,\n    "muscleGroup": "back"\n}'),
   ('squats 100kg 5x5',
    '{\n    "name": "squats",\n    "sets": 5,\n    "reps": 5,\n    "weight": 10

In [35]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)

In [36]:
handlers

{'recordExercise': {'handler': 'import { db } from "../db";\nimport { exercisesTable, exerciseHistoryTable } from \'../db/schema/application\';\n\nconst handle = async (exercise: Exercise): Promise<void> => {\n    // First, check if the exercise already exists\n    const existingExercise = await db\n        .select()\n        .from(exercisesTable)\n        .where(eq(exercisesTable.name, exercise.name))\n        .limit(1);\n\n    let exerciseId: number;\n\n    if (existingExercise.length === 0) {\n        // Insert new exercise if it doesn\'t exist\n        const result = await db.insert(exercisesTable).values({\n            name: exercise.name,\n            sets: exercise.sets,\n            reps: exercise.reps,\n            weight: exercise.weight ?? null,\n            muscle_group: exercise.muscleGroup\n        }).returning({ id: exercisesTable.id });\n        \n        exerciseId = result[0].id;\n    } else {\n        exerciseId = existingExercise[0].id;\n    }\n\n    // Record the e

In [37]:
with open("core/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [38]:
hdl_key = "recordExercise"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [39]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { exercisesTable, exerciseHistoryTable } from '../db/schema/application';

const handle = async (exercise: Exercise): Promise<void> => {
    // First, check if the exercise already exists
    const existingExercise = await db
        .select()
        .from(exercisesTable)
        .where(eq(exercisesTable.name, exercise.name))
        .limit(1);

    let exerciseId: number;

    if (existingExercise.length === 0) {
        // Insert new exercise if it doesn't exist
        const result = await db.insert(exercisesTable).values({
            name: exercise.name,
            sets: exercise.sets,
            reps: exercise.reps,
            weight: exercise.weight ?? null,
            muscle_group: exercise.muscleGroup
        }).returning({ id: exercisesTable.id });
        
        exerciseId = result[0].id;
    } else {
        exerciseId = existingExercise[0].id;
    

In [40]:
print(handlers["suggestWorkout"]["handler"])

import { db } from "../db";
import { 
    exercisesTable, 
    equipmentTable, 
    exerciseHistoryTable 
} from '../db/schema/application';
import { eq, and, desc } from 'drizzle-orm';

const handle = async (
    duration: number, 
    equipment: { name: string, type: string }[], 
    targetMuscle: string
): Promise<{ name: string, sets: number, reps: number, weight?: number, muscleGroup: string }[]> => {
    // Get available equipment IDs
    const equipmentIds = await db
        .select({ id: equipmentTable.id })
        .from(equipmentTable)
        .where(
            and(
                ...equipment.map(e => 
                    and(
                        eq(equipmentTable.name, e.name),
                        eq(equipmentTable.type, e.type)
                    )
                )
            )
        );

    // Get suitable exercises for the target muscle group with available equipment
    const exercises = await db
        .select({
            name: exercisesTable.name,
 

In [41]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [42]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: String\n    ingredients: Ingredient[]\n}\n\nmodel Ingredient {\n    name: String\n    calories: Int\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: Date, to: Date): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [43]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [44]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [45]:
from shutil import copytree, ignore_patterns

In [46]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [47]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [9]:
from core.stages import router

In [79]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [49]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise with details like name, sets, reps, weight (optional), and muscle group targeted. Use this when a user wants to log one specific exercise.',
  'examples': ['I did 3 sets of 12 bench presses with 135 pounds',
   'Just finished 4 sets of pull-ups, 8 reps each',
   'Completed leg press: 3 sets, 10 reps at 225 pounds']},
 {'name': 'logWorkoutSession',
  'description': 'Records a complete workout session including multiple exercises, duration, and date. Use this when a user wants to log their entire workout routine or multiple exercises at once.',
  'examples': ['I want to record my exercise routine for today',
   'Let me log my 45-minute workout from this morning',
   'Need to save my chest and triceps workout from yesterday']},
 {'name': 'suggestWorkout',
  'description': 'Suggests exercises based on available equipment, desired workout duration, and target muscle group. Use this when a user asks for workout recommend

In [83]:
from core.stages import typespec

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [84]:
typespec

{'reasoning': 'For a gym tracking bot, I expect users to send messages like:\n"I did 3 sets of bench press with 80kg today" -> recordExercise()\n"Show my progress for bench press" -> getProgress()\n"Suggest a 30-minute workout for chest using dumbbells" -> suggestRoutine()\n\nKey entities needed:\n- Exercise (name, sets, reps, weight)\n- Equipment (type of gear available)\n- Workout (collection of exercises with duration)\n- Progress tracking (exercise history with dates)\n\nThe interface needs to:\n1. Record individual exercises\n2. Track progress over time\n3. Generate workout suggestions based on constraints\n4. List available equipment',
 'typespec_definitions': 'model Exercise {\n    name: string;\n    sets: integer;\n    reps: integer;\n    weight: float;\n    date: utcDateTime;\n}\n\nmodel Equipment {\n    name: string;\n    category: string;\n}\n\nmodel WorkoutPlan {\n    exercises: Exercise[];\n    duration: duration;\n    difficulty: string;\n    equipment: Equipment[];\n}\n\

In [87]:
import subprocess

# Write TypeSpec definitions to file
with open('./test/tsp_schema/notebook.tsp', 'w') as f:
    f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        
    f.write(typespec["typespec_definitions"])

result_tsp = subprocess.run(['tsp', 'compile', 'notebook.tsp'], 
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            cwd='./test/tsp_schema/.')
print(result_tsp.stdout.decode())
print(result_tsp.stderr.decode())

TypeSpec compiler v0.64.0

Compilation completed successfully.

No emitter was configured, no output was generated. Use `--emit <emitterName>` to pick emitter or specify it in the TypeSpec config.




In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
